In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
input_folder = 'C:/Users/Davide/workspace/SoccerData/Input/'
output_folder = 'C:/Users/Davide/workspace/SoccerData/Output/'

events = pd.read_json(input_folder +'events.json')
players = pd.read_json(input_folder +'players.json')
matches = pd.read_json(input_folder +'matches.json')
teams = pd.read_json(input_folder +'teams.json')

In [3]:
players['role_teo'] = [i['code3'] for i in players.role]

In [4]:
events.head()

,eventId,eventName,eventSec,id,matchId,matchPeriod,playerId,positions,subEventId,subEventName,tags,teamId
0,8,Pass,2.530536,180423957,2575959,1H,8327,"[{'y': 52, 'x': 49}, {'y': 44, 'x': 43}]",85,Simple pass,[{'id': 1801}],3158
1,8,Pass,3.768418,180423958,2575959,1H,20438,"[{'y': 44, 'x': 43}, {'y': 17, 'x': 36}]",85,Simple pass,[{'id': 1801}],3158
2,7,Others on the ball,4.868265,180423959,2575959,1H,8306,"[{'y': 17, 'x': 36}, {'y': 56, 'x': 78}]",72,Touch,[],3158
3,1,Duel,8.114676,180423960,2575959,1H,8306,"[{'y': 56, 'x': 78}, {'y': 15, 'x': 64}]",11,Ground attacking duel,"[{'id': 504}, {'id': 703}, {'id': 1801}]",3158
4,1,Duel,8.647892,180423961,2575959,1H,8306,"[{'y': 15, 'x': 64}, {'y': 15, 'x': 72}]",11,Ground attacking duel,"[{'id': 503}, {'id': 703}, {'id': 1801}]",3158


In [5]:
# Integrate positions as 4 new columns
extr_pos = lambda l: [v for d in l for v in d.values()]

events[['y_start', 'x_start', 'y_end', 'x_end']] = pd.DataFrame(events.positions.apply(extr_pos).tolist(),
                                                                index=events.index)

In [6]:
def mean_and_dispersion_pos(series_pos):
    """This function receives a Series of list of dicts and computes the x mean, the y mean and the
        mean squared distance. Returns a pandas DataFrame"""
    ys = np.array([line[0]['y'] for line in series_pos.values])  # for d in line
    xs = np.array([line[0]['x'] for line in series_pos.values])  # for d in line
    y_mean = ys.mean()
    x_mean = xs.mean()
    dist = np.sqrt(np.power(ys - y_mean, 2) + np.power(xs - x_mean, 2))
    return pd.DataFrame([[x_mean, y_mean, dist.mean()]],
                        columns=['posizione_media_x', 'posizione_media_y', 'standard_deviation'])

def extract_time_delta(series_time):
    """Computes difference between subsequent events"""
    series_time = series_time.sort_values()
    return np.sum(series_time - series_time.shift(1))

In [72]:
# Avg position and dispersion

# result1 = events.groupby(['matchId', 'playerId','matchPeriod']).positions.apply(mean_and_dispersion_pos)
# result1.index = result1.index.droplevel(3)
# result1.reset_index().merge(players[['wyId', 'height', 'weight']], left_on=['playerId'], right_on=['wyId'],
#                            how='left')\
# .drop('wyId', axis=1)
#result1.index = result1.index.droplevel(3)

In [7]:
result2 = events.groupby(['matchId', 'playerId', 'matchPeriod'], as_index=False).agg(
    {'eventSec': extract_time_delta}).merge(
    events.groupby(['matchId', 'playerId', 'matchPeriod'], as_index=False).agg({'id': 'nunique'}),
    on=['matchId', 'playerId', 'matchPeriod']).set_index(['matchId', 'playerId', 'matchPeriod']).assign(
    tempo_medio_tra_eventi=lambda x: x.eventSec / x.id).loc[:, ['tempo_medio_tra_eventi']]


In [8]:
# Rate eventi

result2 = events\
.groupby(['matchId','playerId', 'matchPeriod'], as_index=False).agg({'eventSec': extract_time_delta})\
.merge(events.groupby(['matchId','playerId','matchPeriod'], as_index=False).agg({'id': 'nunique'}), on=['matchId', 'playerId','matchPeriod'])\
.set_index(['matchId', 'playerId','matchPeriod'])\
.assign(tempo_medio_tra_eventi=lambda x: x.eventSec/x.id).loc[:, ['tempo_medio_tra_eventi']]

In [62]:
# result2.reset_index().loc[result2.reset_index().playerId != 0]#.to_csv('rate_event_time.csv', index=False)

In [9]:
def make_list(list_dict, key = 'id'):
    if len(list_dict) ==0:
        return []
    return [i['id'] for i in list_dict]

events['taglist'] = events.tags.apply(make_list)

In [10]:
def select_from_event(df, Id=None, subId=None, tag = None):
    tag_list =[tag in i for i in df.taglist] if tag is not None else [not tag in i for i in df.taglist] 
    id_list = Id if Id is not None else df.eventId
    subid_list = subId if subId is not None else df.subEventId
        
    return df[(df.eventId == id_list) & tag_list & (df.subEventId == subid_list)]

In [61]:
# Find level of ambidestrism

# select_from_event(events, tag=401).groupby(['playerId'], as_index=False)\
# .agg({'id': np.size}).rename(columns={'id': 'left'})\
# .merge(select_from_event(events, tag=402).groupby(['playerId'], as_index=False)\
#       .agg({'id': np.size}).rename(columns={'id': 'right'}),on=['playerId'], how='outer').fillna(0)\
# .assign(ambidestro=lambda x: (x.left - x.right).abs()/ (x.left + x.right)).iloc[:, [0, 3]]#.to_csv('ambid.csv',
#                                                                                                 # index=False)

In [11]:
events['has_doublepos'] = events.positions.apply(lambda l: True if len(l) >= 2 else False)
events.groupby('eventName').has_doublepos.sum() * 100 / events.groupby('eventName').id.size()
sub = events[events.has_doublepos].copy()


In [12]:

def angle_check(pos_tup):
    y_end = pos_tup[0]
    y_start = pos_tup[1]
    x_end = pos_tup[2]
    x_start = pos_tup[3]
    
    deltay = y_end - y_start
    deltax = x_end - x_start
    if deltay >=0:
        if deltax == 0.:
            return np.pi/2
        elif deltax >0:
            return np.arctan(deltay/deltax)
        else:
            return np.pi - abs(np.arctan(deltay/deltax))
    else:
        if deltax == 0:
            return 3*np.pi/2
        elif deltax > 0:
            return 2*np.pi - abs(np.arctan(deltay/deltax))
        else:
            return np.pi + abs(np.arctan(deltay/deltax))

events.loc[events.has_doublepos, 'angle'] = events.loc[events.has_doublepos, ['y_end',
                                                                             'y_start',
                                                                             'x_end',
                                                                             'x_start']].apply(angle_check, axis=1)

In [15]:
events.loc[events.has_doublepos, 'angle'] = events.loc[events.has_doublepos, ['y_end',
                                                                          'y_start',
                                                                          'x_end',
                                                                          'x_start']].apply(angle_check, axis=1)
angdf = events.groupby(['matchId', 'playerId', 'matchPeriod'], as_index=False).agg({'angle': [np.mean, np.std]})
angdf.columns = ['matchId', 'playerId', 'matchPeriod','angle_mean', 'angle_std']

In [16]:
angdf

,matchId,playerId,matchPeriod,angle_mean,angle_std
0,2575959,0,1H,3.823606,1.118371
1,2575959,0,2H,3.669975,0.862645
2,2575959,114,1H,3.311297,1.768162
3,2575959,114,2H,2.872542,1.789314
4,2575959,625,2H,3.720082,2.160301
5,2575959,3463,2H,2.993800,2.683814
6,2575959,3475,1H,2.490079,1.503525
7,2575959,3475,2H,2.837451,1.444033
8,2575959,8306,1H,2.542924,1.903050
9,2575959,8306,2H,2.531475,2.201397
